
# Projet Maison 2

In [179]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [180]:
# chargement et traitement des données
GEO = pd.read_csv("../Kit-Big-Data/session2/correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")

# A COMPLETER avec les colonnes
# - lat, lon : latitude et longitude des communes
# - cp_ville : Code Postal + " " + "Commune"
GEO[['lat', 'lon']] = GEO["geo_point_2d"].str.split(', ', 1, expand=True).astype("float")
GEO["cp_ville"] = GEO["Code Postal"] + " " + GEO["Commune"]


**Partie A**

- Compléter le chargement des données en ajoutant au dataframe `GEO`
    - les colonnes "lat" et "lon" avec la latitude et la longitude des communes
    - une colonne "cp_ville" avec le Code Postal + un espace + et le nom de la Commune
- Ecrire une fonction `search_city(lat, lon)` qui retourne le "cp_ville" de la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ecrire une fonction `dms2dec(deg, min, sec)` qui convertit les degrés, minutes, secondes en valeur numérique pour pouvoir utiliser la fonction précédente avec un GPS.

In [181]:
GEO['lon'].min()

-61.7795787226

In [182]:
from math import sqrt

def eucl(lat1, lon1, lat2, lon2):
    return (lat2.sub(lat1).pow(2) + lon2.sub(lon1).pow(2)).pow(0.5)

In [183]:
# fonction recherche de ville
def search_city(lat, long):
    id = eucl(lat, long, GEO['lat'], GEO['lon']).idxmin()
    return GEO.loc[id, ['cp_ville']].values[0]

search_city(41.5, 51.1)

'20230 SAN-GIULIANO'

In [184]:
type((GEO['lat'].sub(41,5).pow(2)))

pandas.core.series.Series

In [185]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    dd = deg + mn / 60 + sec / 3600
    return dd

In [186]:
# on applique la fonction à une coordonnée tirée au hasard
np.random.seed(0)
a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = np.random.uniform(a, b)
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
lon = np.random.uniform(a, b)

search_city(lat, lon)

'71330 BOSJEAN'

In [187]:
# à partir de coordonnées GPS précises
search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'91120 PALAISEAU'

**Partie B**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python **json** pour parser les valeurs de la colonne "geo_shape" et mettre le résultat (`Series`) dans la variable `GEO_SHAPE`.
- Ecrire une fonction `get_types()` qui retourne le décompte (`value_counts()`) des valeurs accédées avec la clé "type".
- Ecrire une fonction `get_coordinates_len()` qui retourne le décompte (`value_counts()`) des longueurs des listes accédées avec la clé "coordinates".
- Ecrire une fonction `get_most_complex_city()` qui retourne la commune est constituée du plus grand nombre de polygones ?
- Ecrire une fonction `get_nb_cities_2_polygons()` qui retourne  le nombre de villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- **Facultatif :**
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : on pourra installer la librairie **shapely**, utiliser la classe Polygon de **shapely.geometry**  et la méthode `contains()`. Sur Windows **shapely** peut nécessiter d'installer manuellement la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [188]:
import json

# La variable GEO_SHAPE doit contenir une Serie
# correspondant aux valeurs de la colonne "geo_shape" parsées avec la librairie json
GEO_SHAPE = GEO["geo_shape"].apply(lambda x: json.loads(x))

In [189]:

df = pd.DataFrame(list(GEO_SHAPE.values), index=GEO_SHAPE.index)

In [190]:
# value_counts des valeurs "type"
def get_types():
    return pd.DataFrame(list(GEO_SHAPE.values)).value_counts("type")

In [191]:
# value_counts des longueurs de "coordinates"
def get_coordinates_len():
     return df["coordinates"].apply(lambda x: len(x)).value_counts()

In [192]:
# commune constituée du plus grand nombre de polygones
def get_most_complex_city():
    id = df["coordinates"].apply(lambda x: len(x)).idxmax()
    return GEO["cp_ville"][id]

In [193]:
# commune constituée du plus grand nombre de polygones
def get_nb_cities_2_polygons():
    mask = df.loc[df["type"] == "Polygon"]["coordinates"].apply(lambda x: len(x)) == 2
    return mask.sum()

In [194]:
# tests
import unittest

class Session2Test(unittest.TestCase):
    
    def test_partie_A1(self):
        # on applique la fonction cherche_ville() à une coordonnée tirée au hasard
        np.random.seed(0)
        a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
        lat = np.random.uniform(a, b)
        a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
        lon = np.random.uniform(a, b)

        cp_ville = search_city(lat, lon)
        self.assertEqual(cp_ville, "71330 BOSJEAN")
        
    def test_partie_A2(self):
        # à partir de coordonnées GPS précises
        cp_ville = search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))
        self.assertEqual(cp_ville, "91120 PALAISEAU")
        
    def test_partie_B1(self):
        # check types
        dico = get_types()
        self.assertEqual(dico["Polygon"], 36670)
        self.assertEqual(dico["MultiPolygon"], 72)
        
    def test_partie_B2(self):
        # check coordinates len
        dico = get_coordinates_len()
        self.assertEqual(dico[1], 36660)
        self.assertEqual(dico[2], 80)
       
    def test_partie_B3(self):
        # check most complex city
        cp_ville = get_most_complex_city()
        self.assertEqual(cp_ville, "83400 HYERES")
        
    def test_partie_B4(self):
        # check nb cities 2 polygons
        nb = get_nb_cities_2_polygons()
        self.assertEqual(nb, 10)

In [195]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session2Test)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)
    
run_tests()

test_partie_A1 (__main__.Session2Test) ... ok
test_partie_A2 (__main__.Session2Test) ... ok
test_partie_B1 (__main__.Session2Test) ... ok
test_partie_B2 (__main__.Session2Test) ... ok
test_partie_B3 (__main__.Session2Test) ... ok
test_partie_B4 (__main__.Session2Test) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.171s

OK
